**Author:** J. Žovák, `482857@mail.muni.cz`
Note: inspired by lvd/examples/basic_functionality/client_auth.ipynb

In [ ]:
import chromadb
from chromadb import Settings
import pandas as pd

client = chromadb.HttpClient(host="localhost", port=5000)
client.heartbeat()  

client.get_version()

client.list_collections() 


In [ ]:
n_categories = [4]

index_configuraiton = {
    "lmi:epochs": "[200]",
    "lmi:model_types": "['MLP']",
    "lmi:lrs": "[0.01]",
    "lmi:n_categories": f"{str(n_categories)}",
    "lmi:kmeans": "{'verbose': False, 'seed': 2023, 'nredo': 10}",
}

collections = client.list_collections()
if collections:
    client.delete_collection(collections[0].name)
    
collection_name = "synthetic_collection_2"
collection = client.create_collection(
    name=collection_name,
    metadata=index_configuraiton
)

In [ ]:
csv_file_path = 'data/synthetic_clusters_colored.csv'
data = pd.read_csv(csv_file_path)

In [ ]:
# Use batch upload just to test it out
batch_size = 25
for i in range(0, len(data), batch_size):
    collection.add(
        embeddings=data[['x', 'y']].iloc[i: i + batch_size].values.tolist(),
        metadatas=[{"cluster": cluster} for cluster in data['cluster'].iloc[i: i + batch_size]],
        ids=data['id'].iloc[i: i + batch_size].values.tolist(),
    )

bucket_assignment = collection.build_index()

In [ ]:
query_point = data[data['cluster'] == "purple"][['x', 'y']].iloc[0]

results = collection.query(
    query_embeddings=list(query_point),
    include=["metadatas", 'embeddings', 'distances'],
    where={"cluster": "red"},
    n_results=10,
    n_buckets=1,
    bruteforce_threshold=0.0,
    constraint_weight=0,
    search_until_bucket_not_empty=True, 
)

In [ ]:
results['ids']